In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm

from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

# Kies testing_set is False of True!!

In [2]:
# HIER!!!!!!
testing_set = False

In [35]:
# testdf = pd.read_hdf("./data/corrected_price_testset.hdf")
# traindf = pd.read_hdf("./data/corrected_price.hdf")

# testdf = pd.read_hdf("./data/corrected_price_testset_rollback_and_avg.hdf")

# traindf = pd.read_hdf("./data/corrected_price_rollback_and_avg.hdf")

testdf = pd.read_csv("data/test_set_VU_DM.csv")
traindf = pd.read_csv("data/training_set_VU_DM.csv")

print(len(traindf.columns))
print(len(testdf.columns))

print(traindf.columns)
print(testdf.columns)

54
50
Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7

In [42]:
def calculate_missing_percentage(df):
    return ((df.isna().mean() * 100).sum() / len(df.columns)).round(4)

print("Percentage test: ", calculate_missing_percentage(testdf))
print("Percentage train: ", calculate_missing_percentage(traindf))


print(len(testdf))
print(len(traindf))

print(len(testdf.srch_id.unique()))
print(len(traindf.srch_id.unique()))

print(len(testdf.prop_id.unique()))
print(len(traindf.prop_id.unique()))

Percentage test:  46.3779
Percentage train:  44.7182
4959183
4958347
199549
199795
129438
129113


In [51]:
traindf2 = competitors(traindf)
traindf2 = traindf2.drop(columns=['comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff'])

traindf2 = visitor_history(traindf2)
traindf2 = traindf2.drop(columns=['visitor_hist_starrating', 'visitor_hist_adr_usd', 'gross_bookings_usd', 'srch_query_affinity_score', 'orig_destination_distance'])


print("Percentage train: ", calculate_missing_percentage(traindf2))

Percentage train:  0.7907


In [52]:
testdf2 = competitors(testdf)
testdf2 = testdf2.drop(columns=['comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff'])

testdf2 = visitor_history(testdf2)
testdf2 = testdf2.drop(columns=['visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score', 'orig_destination_distance'])

print("Percentage train: ", calculate_missing_percentage(testdf2))

Percentage train:  0.8835


In [ ]:
def add_score(df):
    """
    Add an importance score based on click_bool and booking_bool
    """
    
    # every hotel that is clicked on gets an importance score of 1
    df["importance"] = df["click_bool"]
    
    # every hotel that is booked gets an importance score of 5 
    df["importance"][df["booking_bool"] == 1] = 5
        
    return df

In [ ]:
def estimate_pos(traindf, testdf):
    
    # Only select non-random 
    dest_dict = traindf.loc[traindf["random_bool"] == 0]
    
    dest_dict = traindf.groupby(["srch_destination_id", "prop_id"]).agg(
        {"position": "mean"}
    )
    dest_dict = dest_dict.rename(
        index=str, columns={"position": "estimated_position"}).reset_index()
    
    dest_dict["srch_destination_id"] = (
        dest_dict["srch_destination_id"].astype(str).astype(int)
    )
    dest_dict["prop_id"] = (
        dest_dict["prop_id"].astype(str).astype(int))
    
    dest_dict["estimated_position"] = (1 / dest_dict["estimated_position"])
    
    testdf = testdf.merge(dest_dict, how="left", on=["srch_destination_id", "prop_id"])
    
    traindf = traindf.merge(dest_dict, how="left", on=["srch_destination_id", "prop_id"])
    
    return testdf, traindf

def estimate_importance(traindf, testdf):
    
    # Only select non-random 
    dest_dict = traindf.loc[traindf["random_bool"] == 0]
    
    dest_dict = traindf.groupby(["srch_destination_id", "prop_id"]).agg(
        {"importance": "mean"}
    )
    dest_dict = dest_dict.rename(
        index=str, columns={"importance": "estimated_importance"}).reset_index()
    
    dest_dict["srch_destination_id"] = (
        dest_dict["srch_destination_id"].astype(str).astype(int)
    )
    dest_dict["prop_id"] = (
        dest_dict["prop_id"].astype(str).astype(int))
    
    dest_dict["estimated_importance"] = (1 / dest_dict["estimated_importance"])
    
    testdf = testdf.merge(dest_dict, how="left", on=["srch_destination_id", "prop_id"])
    
    traindf = traindf.merge(dest_dict, how="left", on=["srch_destination_id", "prop_id"])
    
    return testdf, traindf



traindf = add_score(traindf)
print("added score")
    
testdf, traindf = estimate_pos(traindf, testdf)
testdf, traindf = estimate_importance(traindf, testdf)


In [ ]:
def normalize_features(input_df, group_key, target_column, take_log10=False):

    # for numerical stability
    epsilon = 1e-4
    if take_log10:
        input_df[target_column] = np.log10(input_df[target_column] + epsilon)
    methods = ["mean", "std"]

    df = input_df.groupby(group_key).agg({target_column: methods})

    df.columns = df.columns.droplevel()
    col = {}
    for method in methods:
        col[method] = target_column + "_" + method

    df.rename(columns=col, inplace=True)
    df_merge = input_df.merge(df.reset_index(), on=group_key)
    df_merge[target_column + "_norm_by_" + group_key] = (
        df_merge[target_column] - df_merge[target_column + "_mean"]
    ) / df_merge[target_column + "_std"]
    df_merge = df_merge.drop(labels=[col["mean"], col["std"]], axis=1)

    return df_merge

In [ ]:
if testing_set is True:
    filename = "./data/test_clean.hdf"
    traindf = testdf
else:
    filename = "./data/traindf_clean.hdf"
    
    traindf = traindf[traindf.random_bool == 0]

In [ ]:
def downsampling(df):
    """
    Balance classes in trainingset, based on click_bool (not booking_bool)
    """
    
    length = len(df.loc[df.importance == 5])
    length1 = len(df.loc[df.importance == 1])

    
    # Get 50% of data with importance of 5 or 1
    clicks = df[df.importance == 5].index
    randoms = np.random.choice(clicks, length , replace=False)
    click_sample = df.loc[randoms]
    

    not_click = df[df.importance == 1].index
    random_indices = np.random.choice(not_click, length1, replace=False)
    not_click_sample = df.loc[random_indices]
    print(len(random_indices))
    
    not_click = df[df.importance == 0].index
    random_indices = np.random.choice(not_click, length1, replace=False)
    not_click_sample2 = df.loc[random_indices]
    print(len(random_indices))

    df_new = pd.concat([click_sample, not_click_sample, not_click_sample2], axis=0)
    
    print("Percentage of not click impressions: ", len(df_new[df_new.importance == 0])/len(df))
    print("Percentage of click impression: ", len(df_new[df_new.importance != 0])/len(df))
    
    return df_new


In [17]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """

    # we say that there is no competitor with a lower price
    df["competitor_lower"] = 0 #competitor_bools
    
    # comp1rate = 1 if price is lower
    df["competitor_lower"][df["comp1_rate"] == 1] = 1
    df["competitor_lower"][df["comp2_rate"] == 1] = 1
    df["competitor_lower"][df["comp3_rate"] == 1] = 1
    df["competitor_lower"][df["comp4_rate"] == 1] = 1
    df["competitor_lower"][df["comp5_rate"] == 1] = 1
    df["competitor_lower"][df["comp6_rate"] == 1] = 1
    df["competitor_lower"][df["comp7_rate"] == 1] = 1
    df["competitor_lower"][df["comp8_rate"] == 1] = 1
    
    # we say at first there is no competitor hotel available
    df["competitor_available"] = 0
    
    # availability bool = 1 if there if the competitor and expedia are available
    df["competitor_available"][df["comp1_inv"] == 1] = 1
    df["competitor_available"][df["comp2_inv"] == 1] = 1
    df["competitor_available"][df["comp3_inv"] == 1] = 1
    df["competitor_available"][df["comp4_inv"] == 1] = 1
    df["competitor_available"][df["comp5_inv"] == 1] = 1
    df["competitor_available"][df["comp6_inv"] == 1] = 1
    df["competitor_available"][df["comp7_inv"] == 1] = 1
    df["competitor_available"][df["comp8_inv"] == 1] = 1

    return df

In [18]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # most visitors haven't visited a hotel yet
    df["visited_before"] = 0
    
    # where there is a history field filled in, visited_before is turned into 21
    df["visited_before"][df["visitor_hist_starrating"].notna() | df["visitor_hist_adr_usd"].notna()] = 1
    
    return df

In [ ]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    Also add review/quality to the df.
    """
    
    df["price_quality"] = np.nan
    df["price_review"] = np.nan
    
    df["price_quality"][df["price_correction"].notna() & df["prop_starrating"].notna() & df["prop_starrating"] != 0] = df["price_correction"] / df["prop_starrating"]
    df["price_review"][df["price_correction"].notna() & df["prop_review_score"].notna() & df["prop_review_score"] != 0] = df["price_correction"] / df["prop_review_score"]

    # Replace missing values with median
#     df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df


In [ ]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    columnames = list(df.columns)
    columnames.extend(["avg_price_propid", "std_avg_price_propid", "amount_hotels", 
                       "avg_price_propid_after", "std_avg_price_propid_after"])
    
    df = df.reindex(columns=columnames)
    
    # copy prices (for now, at the end we will just update the price i suppose)
    df["price_correction"] = df["price_usd"]
    
    print("Made all extra columns")
    display(df[["avg_price_propid", "std_avg_price_propid", "amount_hotels", 
                       "avg_price_propid_after", "std_avg_price_propid_after", "price_correction"]])
    
    amount = 0
    
    propids = list(df.prop_id.unique())
    
    for prop_id in tqdm(propids, desc="Processing propids:"): 
        # calculate average and standard deviation
        std = df["price_usd"][df["prop_id"] == prop_id].std()
        avg = df["price_usd"][df["prop_id"] == prop_id].mean() 
        
        # count how many times this hotel appears in the dataframe
        df["amount_hotels"][df["prop_id"] == prop_id] = len(df[df["prop_id"] == prop_id])
        
        # put average and standard deviation in dataframe
        df["avg_price_propid"][df["prop_id"] == prop_id] = avg
        df["std_avg_price_propid"][df["prop_id"] == prop_id] = std
        
        # If std is high, correct for number of nights
        if std > 50:
            amount += 1
            df["price_correction"][df["prop_id"] == prop_id] = df["price_usd"][df["prop_id"] == prop_id] / df["srch_length_of_stay"][df["prop_id"] == prop_id]
        
        # for now separate columns so we can compare
        df["avg_price_propid_after"][df["prop_id"] == prop_id] = df["price_correction"][df["prop_id"] == prop_id].mean()
        df["std_avg_price_propid_after"][df["prop_id"] == prop_id] = df["price_correction"][df["prop_id"] == prop_id].std()
            
    print("Amount of properties with std > 50: ", amount)
    
    # Ik kreeg hier een error sorry
#     print("Correcting ", len(df[df["std_avg_price_propid"] > 50]), " property prices")
    
    return df

In [ ]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values with median
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    df["total_loc_score"] = df["prop_location_score2"] + df["prop_location_score1"]

    df["prop_location_score1"].fillna(-1, inplace=True)
    df["prop_location_score2"].fillna(0, inplace=True)
    
    df["prop_starrating"].fillna(0, inplace=True)
    
    
    return df

In [ ]:
def price_rank(df):
    """
    Add the rank for every prop_id within each srch_id
    """
    df["price_rank"] = df.groupby("srch_id")["price_usd"].rank()
    
    return df

In [ ]:
def locationscore_rank(df):
    """
    Add the rank for every location score within each srch_id
    """
    df["locationscore2_rank"] = df.groupby("srch_id")["prop_location_score2"].rank()
    df["locationscore1_rank"] = df.groupby("srch_id")["prop_location_score1"].rank()
    
    return df

In [ ]:
def starrating(df):
    """
    Also add starrating rank and mean starrating for each property
    """
    
    df["starrating_rank"] = df.groupby("srch_id")["prop_starrating"].rank()
    
    
    df["mean_rating_propid"] = df.groupby("prop_id")["prop_starrating"].transform('mean')
    
    return df

In [ ]:
def price_diff_hist(df):
    
    df["prop_log_historical_prices"] =  10 ** df["prop_log_historical_price"]
    
    df["price_diff_hist"] = df["price_usd"] - df["prop_log_historical_prices"]
    df["diff_price_srchid"] = df["price_correction"] - df['avg_price_propid_after']
    df["diff_price_propid"] = df["price_correction"] - df['avg_price_srchid']
    
    return df

### Aparte functies aanroepen voor de kolommen die je erbij wilt

In [ ]:
# NORMALIZATION
traindf = normalize_features(traindf, group_key="prop_id", target_column="price_usd", take_log10=False)
print("Normalized price_usd by prop_id")

traindf = normalize_features(traindf, group_key="srch_id", target_column="prop_starrating")
print("Normalized prop_starrating by srch_id")

traindf = normalize_features(traindf, group_key="srch_id", target_column="prop_location_score2")
print("Normalized prop_locationscore2 by srch_id")

In [ ]:
# traindf = competitors(traindf)
# print("cleaned competitors")

# traindf = visitor_history(traindf)
# print("cleaned visitor history")

traindf = process_remaining_cols(traindf)
print("did remaining columns")

traindf = price_quality(traindf)
print("cleaned price quality")

traindf = price_rank(traindf)
print("added a rank of price per search_id")

traindf = locationscore_rank(traindf)
print("added a rank of location score search_id")

traindf = starrating(traindf)
print("added starrating and mean of prop_ids")

traindf = price_diff_hist(traindf)
print("added history of price info")

# Add relevant columns    
if testing_set is False:
    
    df = traindf[["prop_id", 
                  "srch_id", 
                  "position", 
                  "price_quality", 
                  "price_review",
                  "click_bool", 
                  "booking_bool",  
                  "price_usd",
#                   "prop_location_score1", 
                  "prop_location_score2",
                  "avg_price_propid", 
                  "std_avg_price_propid", 
                  "amount_hotels", 
                  "avg_price_propid_after", 
                  "price_rank",
#                   "price_correction",
                  "locationscore2_rank",
                  "locationscore1_rank",
                  "starrating_rank",
                  "price_diff_hist",
                  "diff_price_srchid",
                  "diff_price_propid",
                  "total_loc_score",
                  "estimated_position",
                   "price_usd_norm_by_prop_id",
                  "prop_starrating_norm_by_srch_id",
                  "prop_location_score2_norm_by_srch_id",
                  "estimated_importance"
                 ]]
    
    df = add_score(df)
    print("added score")
    
    # Balance data to 50% importance score or 1 or 5 and 0 
#     df = downsampling(df)
#     print("Downsampled data")
    
else:
    
    # df without click_bool, booking_bool and position
    df = traindf[["prop_id", 
                  "srch_id", 
                  "price_quality", 
                  "price_review",
                  "price_usd",
#                   "prop_location_score1", 
                  "prop_location_score2",
                  "avg_price_propid", 
                  "std_avg_price_propid", 
                  "amount_hotels", 
                  "avg_price_propid_after", 
                  "price_rank",
#                   "price_correction",
                  "locationscore2_rank",
                  "locationscore1_rank",
                  "starrating_rank",
                  "price_diff_hist",
                  "diff_price_srchid",
                  "diff_price_propid",
                  "total_loc_score",
                  "estimated_position",
                  "price_usd_norm_by_prop_id",
                  "prop_starrating_norm_by_srch_id",
                  "prop_location_score2_norm_by_srch_id",
                  "estimated_importance"
                 ]]

In [ ]:
traindf.head()

In [ ]:
display(df.head())
display(df.describe())

In [ ]:
# df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

## Totale dataset gepreprocessed

In [ ]:
print(len(df))

In [ ]:
# save the dataframe if it does not exist yet
# if not os.path.exists(filename):
#     df.to_hdf(filename, key="df", format="table")
df.to_hdf(filename, key="df", format="table")